In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from surprise import SVD
from surprise import Dataset
import surprise
from surprise.model_selection import cross_validate
from collections import defaultdict

display(HTML(data="""
<style>
    div#notebook-container    { width: 90%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 90%; }
</style>
"""))

In [3]:
df_usr_rating =pd.read_csv('C://Users//rupadhyay//Desktop//ML//Analytics Summit Bibliotheca E Book subscription prediction//Data//df_rating.csv',encoding='latin1')

In [89]:
df_usr_rating.head(50)

,UserID,BookID,U_RATING
0,654353,170387,0.000000
1,611604,170387,0.000000
2,674336,170387,0.000000
3,626189,170387,0.000000
4,619059,170387,0.000000
5,643738,170387,0.000000
6,666193,170387,0.000000
7,648147,170387,0.000000
8,615974,170387,0.000000
9,667773,170387,0.000000


In [40]:
df_usr_rating['U_RATING'] = df_usr_rating['U_RATING'].replace({0.0:np.nan, 0:np.nan})

In [49]:
df_usr_rating['U_RATING']=(df_usr_rating['U_RATING']-df_usr_rating['U_RATING'].mean())/df_usr_rating['U_RATING'].std()

In [42]:
df_usr_rating['U_RATING'] = df_usr_rating['U_RATING'].abs()

In [48]:
df_usr_rating['UserID'] = df_usr_rating['UserID'].apply(str)

In [49]:
df_usr_rating['BookID'] = df_usr_rating['BookID'].apply(str)

In [50]:
df_usr_rating.dtypes

UserID       object
BookID       object
U_RATING    float64
dtype: object

In [4]:
df_usr_rating['U_RATING'].min()

0.0

In [5]:
df_usr_rating['U_RATING'].max()

5.0

In [53]:
reader = surprise.Reader(rating_scale = (0.0006635953010834748, 5.385724829405537))
data = surprise.Dataset.load_from_df(df_usr_rating,reader)

In [54]:
alg = surprise.SVDpp()
output = alg.fit(data.build_full_trainset())

In [57]:
pred = alg.predict(uid='606494',iid='170387')
score = pred.est
print(score)

5.385724829405537


In [61]:
iids = df_usr_rating['BookID'].unique
iids606494 = df_usr_rating.loc[df_usr_rating['UserID']=='606494', 'BookID']
iids_to_pred = np.setdiff1d(iids,iids606494)

In [79]:
#testset = [[606494, iid, 5.385724829405537] for iid in iids_to_pred]
#predictions = alg.test(testset)
#predictions[0]

In [84]:
param_grid = {'lr_all': [.001, .01], 'reg_all' : [.1, .5]}
gs = surprise.model_selection.GridSearchCV(surprise.SVDpp, param_grid, measures=['rmse','mae'], cv=3)
gs.fit(data)

print(gs.best_params['rmse'])

{'lr_all': 0.001, 'reg_all': 0.1}


In [86]:
alg = surprise.SVDpp(lr_all = 0.001 , reg_all=0.1 )
output = surprise.model_selection.cross_validate(alg, data, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    nan     nan     nan     nan     nan     nan     nan     
MAE (testset)     nan     nan     nan     nan     nan     nan     nan     
Fit time          195.72  200.30  200.13  205.15  204.49  201.16  3.42    
Test time         4.08    3.91    4.86    4.27    4.25    4.27    0.32    


### USING SVD++

In [6]:
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n



In [7]:

# First train an SVD algorithm 
reader = surprise.Reader(rating_scale = (0, 5))
data = surprise.Dataset.load_from_df(df_usr_rating,reader)
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)



In [8]:
top_n

defaultdict(list,
            {654353: [(118603, 1.7068450797154897),
              (126647, 1.3996537036382932),
              (178261, 1.37063738860097),
              (154804, 1.3347487464461065),
              (143688, 1.2849237364673427),
              (103794, 1.2395151194738254),
              (129631, 1.2017869283452893),
              (162832, 1.157881147765162),
              (157022, 1.1512323069163863),
              (170816, 1.1316180262060782)],
             611604: [(118603, 1.4517209972962561),
              (174056, 1.382084569384261),
              (154804, 1.317489511031753),
              (143688, 1.3107241988934888),
              (150279, 1.2717188734221891),
              (170816, 1.0603683898004355),
              (162832, 1.0371847086213641),
              (173567, 1.0184885101323164),
              (121582, 0.9943734417869385),
              (133211, 0.9813254149215807)],
             674336: [(118603, 1.354739533019306),
              (143688, 1.2158897784243

In [9]:
# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

654353 [118603, 126647, 178261, 154804, 143688, 103794, 129631, 162832, 157022, 170816]
611604 [118603, 174056, 154804, 143688, 150279, 170816, 162832, 173567, 121582, 133211]
674336 [118603, 143688, 102688, 170816, 154804, 129631, 162671, 173567, 162832, 133211]
626189 [118603, 154804, 143688, 150279, 162832, 170816, 174056, 131325, 126647, 178261]
619059 [118603, 154804, 126647, 143688, 174056, 170816, 162832, 131325, 175147, 122549]
643738 [150279, 143688, 174056, 118603, 154804, 126647, 175147, 138507, 156981, 170816]
666193 [118603, 150279, 154804, 143688, 175147, 162832, 174056, 162671, 105481, 157022]
648147 [118603, 154804, 143688, 170816, 156981, 129631, 138507, 178261, 133211, 162671]
615974 [118603, 154804, 143688, 178261, 162671, 174056, 131325, 175147, 162832, 136063]
667773 [118603, 154804, 143688, 138507, 174056, 175147, 162832, 156940, 157022, 105481]
605935 [118603, 143688, 154804, 170816, 126647, 175147, 178261, 138507, 174056, 121582]
604933 [118603, 178261, 143688, 

620850 [118603, 154804, 143688, 162832, 170816, 126647, 103794, 174056, 102688, 105481]
636700 [154804, 118603, 143688, 178261, 133211, 174056, 150279, 118065, 126647, 162832]
662308 [118603, 154804, 143688, 178261, 174056, 126647, 150279, 138507, 127590, 112248]
655323 [154804, 118603, 150279, 143688, 170816, 131325, 174056, 162832, 102688, 126647]
630307 [118603, 150279, 178261, 154804, 175147, 126647, 143688, 156863, 170816, 174056]
664894 [118603, 154804, 143688, 178261, 138507, 161044, 175147, 156981, 125941, 126647]
653143 [118603, 154804, 143688, 178261, 121582, 170816, 173567, 167230, 126647, 103794]
652735 [118603, 126647, 150279, 143688, 138507, 154804, 156863, 162832, 172288, 105481]
618276 [118603, 143688, 102688, 154804, 162671, 170816, 103794, 174056, 150279, 138507]
659165 [118603, 143688, 154804, 175147, 126647, 170816, 162832, 129631, 121582, 131325]
613032 [118603, 154804, 143688, 174056, 170816, 175147, 150279, 131325, 129631, 126647]
657589 [118603, 143688, 154804, 

643544 [118603, 143688, 178261, 170816, 126647, 154804, 103794, 174056, 162832, 135103]
620842 [150279, 118603, 154804, 102688, 143688, 131325, 136063, 157022, 162832, 175147]
670221 [118603, 150279, 126647, 125941, 131325, 173567, 162832, 174056, 133211, 175147]
616213 [118603, 154804, 143688, 174056, 178261, 126647, 162832, 138507, 156981, 150279]
665788 [154804, 118603, 150279, 143688, 174056, 105481, 175147, 162671, 162832, 126647]
659971 [118603, 143688, 174056, 154804, 178563, 138507, 150279, 156981, 162671, 162832]
618780 [118603, 154804, 143688, 175147, 131325, 126647, 142714, 105481, 162832, 174056]
638217 [118603, 143688, 175147, 154804, 129631, 102688, 162832, 170816, 156981, 138507]
672789 [154804, 118603, 150279, 174056, 103794, 102688, 162832, 136063, 157022, 143688]
603804 [118603, 143688, 150279, 174056, 178261, 136063, 157022, 131325, 103794, 170816]
601319 [118603, 154804, 143688, 162671, 174056, 126647, 131325, 175147, 103794, 162832]
657864 [118603, 154804, 143688, 

602338 [143688, 118603, 154804, 126647, 150279, 129631, 131325, 162671, 174056, 178261]
607392 [118603, 154804, 175147, 150279, 143688, 178261, 174056, 131325, 133160, 103794]
627672 [154804, 118603, 143688, 131325, 162832, 175147, 105481, 173567, 150279, 178261]
621288 [118603, 154804, 178261, 138507, 129371, 164397, 173567, 143688, 103794, 175147]
629655 [118603, 154804, 143688, 178261, 150279, 175147, 174056, 100148, 126647, 138590]
638027 [118603, 150279, 154804, 143688, 162832, 174056, 121582, 103794, 118065, 102688]
672816 [154804, 118603, 143688, 157022, 102688, 150279, 178261, 126647, 121582, 156981]
607914 [118603, 143688, 154804, 174056, 150279, 170816, 126647, 102688, 103794, 131325]
612485 [118603, 178261, 143688, 126647, 150279, 175147, 174056, 138507, 162832, 162671]
609698 [118603, 154804, 126647, 129631, 143688, 170816, 174056, 150279, 121906, 131325]
602810 [118603, 154804, 143688, 126647, 170816, 174056, 178261, 129631, 131325, 150279]
664257 [118603, 154804, 126647, 

661809 [118603, 154804, 143688, 126647, 178261, 170816, 174056, 150279, 121582, 162671]
646182 [118603, 150279, 154804, 143688, 131325, 175147, 174056, 178261, 102688, 162832]
608322 [118603, 143688, 154804, 126647, 156863, 174056, 150279, 121582, 170816, 162832]
612207 [118603, 178261, 143688, 150279, 154804, 174056, 126647, 162671, 156863, 162832]
658876 [118603, 154804, 143688, 138507, 174056, 157022, 175147, 126647, 102688, 170816]
647436 [118603, 154804, 178261, 150279, 121582, 174056, 143688, 102688, 133211, 131325]
624951 [118603, 143688, 170816, 126647, 154804, 103794, 150279, 138507, 148634, 175147]
621412 [118603, 150279, 154804, 143688, 174056, 157022, 175147, 113299, 162245, 156940]
648816 [143688, 118603, 175147, 154804, 150279, 174056, 105481, 131325, 126647, 170816]
639297 [118603, 154804, 143688, 175147, 178261, 126647, 174056, 103794, 131325, 162671]
669867 [118603, 143688, 138507, 129371, 131325, 150279, 138590, 166232, 126647, 174056]
641579 [118603, 154804, 143688, 

...

648168 [118603, 150279, 178261, 174056, 154804, 175147, 143688, 131325, 112248, 170816]
621416 [118603, 143688, 154804, 126647, 157022, 150279, 138507, 156981, 162832, 175147]
623816 [118603, 143688, 154804, 150279, 175147, 178261, 121582, 173567, 157022, 126647]
659466 [154804, 118603, 143688, 178261, 174056, 162832, 150279, 121582, 105696, 122549]
609944 [118603, 154804, 143688, 175147, 178261, 174056, 138507, 157022, 121582, 126647]
639694 [150279, 118603, 154804, 126647, 174056, 143688, 170816, 102688, 131325, 127193]
643148 [118603, 154804, 143688, 174056, 138590, 178261, 126647, 129631, 150279, 133160]
674866 [118603, 143688, 150279, 154804, 102688, 162832, 157022, 126647, 129631, 131325]
631188 [150279, 178261, 118603, 154804, 105481, 175147, 157022, 170816, 143688, 131325]
609122 [118603, 143688, 154804, 138507, 126647, 131325, 175147, 174056, 103794, 129631]
669347 [154804, 118603, 162832, 150279, 143688, 174056, 102688, 121582, 136063, 126647]
659304 [118603, 154804, 174056, 

662331 [143688, 118603, 154804, 174056, 102688, 156940, 150279, 122549, 178261, 170816]
659635 [118603, 143688, 150279, 162832, 154804, 174056, 175147, 121582, 126647, 162671]
600630 [118603, 154804, 143688, 175147, 170816, 162832, 103794, 126647, 173567, 157022]
638070 [118603, 154804, 126647, 143688, 174056, 118065, 121582, 138507, 150279, 170816]
662570 [150279, 154804, 118603, 143688, 174056, 162671, 178261, 131325, 138507, 129631]
629524 [118603, 154804, 143688, 150279, 175147, 157022, 129631, 173567, 121582, 162671]
612440 [178261, 143688, 154804, 121582, 174056, 170816, 167230, 103794, 162671, 175147]
614831 [118603, 154804, 143688, 131325, 157022, 150279, 162832, 172288, 178261, 173567]
663151 [118603, 178261, 150279, 143688, 174056, 154804, 170816, 126647, 148634, 103794]
667112 [118603, 154804, 178261, 174056, 143688, 126647, 136063, 170816, 156981, 173567]
663781 [154804, 118603, 178261, 121582, 170816, 174056, 103794, 143688, 131325, 126647]
615720 [118603, 143688, 150279, 

619005 [118603, 143688, 154804, 105481, 156863, 157022, 162832, 103794, 131325, 173567]
648581 [118603, 150279, 143688, 154804, 135103, 157022, 136063, 102688, 174056, 162832]
600386 [150279, 143688, 103794, 154804, 126647, 170816, 162832, 131325, 173567, 174056]
654158 [118603, 157022, 143688, 154804, 170816, 156863, 133160, 133211, 173567, 126647]
617373 [118603, 154804, 143688, 105481, 136063, 174056, 118065, 156863, 178261, 170816]
608896 [118603, 143688, 126647, 170816, 174056, 154804, 102688, 157022, 162832, 156981]
637030 [143688, 154804, 174056, 150279, 126647, 162671, 129371, 178261, 164397, 156981]
618219 [118603, 154804, 150279, 178261, 174056, 143688, 162832, 138590, 126647, 147044]
668700 [150279, 118603, 143688, 178261, 154804, 121582, 170816, 103794, 133160, 174056]
642760 [118603, 154804, 131325, 157022, 150279, 126647, 118065, 129371, 174056, 138507]
658776 [118603, 154804, 143688, 121582, 131325, 129631, 150279, 170816, 136063, 125941]
665554 [118603, 150279, 154804, 

660101 [178261, 154804, 118603, 143688, 156863, 129631, 174056, 170816, 162671, 162245]
663401 [118603, 143688, 178261, 150279, 154804, 131325, 174056, 126647, 162832, 122549]
649014 [154804, 143688, 150279, 118603, 178261, 131325, 129631, 174056, 127590, 136063]
673987 [118603, 154804, 143688, 138590, 150279, 138507, 129631, 174056, 102688, 170816]
644768 [118603, 154804, 143688, 126647, 174056, 121906, 170816, 164397, 125941, 122549]
633594 [118603, 154804, 143688, 178261, 150279, 162671, 121582, 103794, 136063, 174056]
672557 [118603, 143688, 154804, 150279, 157022, 126647, 162832, 133211, 132332, 175147]
666485 [118603, 150279, 121582, 143688, 154804, 162671, 174056, 102688, 125228, 103794]
617825 [175147, 118603, 143688, 138507, 154804, 102688, 150279, 126647, 173567, 148634]
647265 [118603, 154804, 102688, 143688, 170816, 126647, 150279, 174056, 122549, 133211]
640292 [118603, 143688, 154804, 126647, 138590, 174056, 170816, 162832, 156863, 157022]
644888 [118603, 102688, 154804, 

619135 [118603, 154804, 143688, 150279, 103794, 126647, 122549, 118065, 131325, 173567]
661428 [118603, 143688, 126647, 154804, 105481, 156940, 162832, 174056, 175147, 131325]
608683 [118603, 154804, 143688, 102688, 175147, 150279, 157022, 131325, 170816, 126647]
623504 [150279, 118603, 154804, 143688, 121582, 127193, 133211, 167230, 162671, 173567]
603296 [118603, 175147, 150279, 143688, 154804, 131325, 170816, 126647, 121582, 174056]
600358 [118603, 150279, 174056, 143688, 154804, 175147, 164397, 157022, 162671, 125941]
603952 [118603, 154804, 150279, 143688, 105481, 157022, 121582, 126647, 162671, 162832]
638321 [118603, 143688, 154804, 102688, 126647, 138507, 174056, 103794, 170816, 162245]
653949 [118603, 143688, 150279, 174056, 170816, 105481, 102688, 126647, 154804, 133211]
622444 [150279, 118603, 143688, 156940, 174056, 175147, 102688, 170816, 121582, 173567]
651792 [154804, 118603, 143688, 175147, 105481, 170816, 126647, 174056, 162832, 173567]
628178 [118603, 143688, 175147, 

In [11]:
user = []

In [13]:
for uid, user_ratings in top_n.items():
    user.append((uid, [iid for (iid, _) in user_ratings]))

In [19]:
user_df2 = pd.DataFrame(user)

In [20]:
#Creating single line csv for predicted ratings
user_df2.to_csv('../Data/predicted_user_rating.csv', index=False)

In [ ]:
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

In [11]:
r_df = pd.DataFrame.from_dict(top_n, orient='index', dtype=None)

In [12]:
r_df.head(50)

,0,1,2,3,4,5,6,7,8,9
654353,"(118603, 1.6989002978456509)","(143688, 1.3498238461561916)","(154804, 1.328749292918841)","(118065, 1.2613676298823124)","(178261, 1.2253094787866505)","(164397, 1.2249707152724674)","(174056, 1.1423179138199284)","(156981, 1.0910678652727652)","(126647, 1.0826178589611293)","(138507, 1.0683220037780603)"
611604,"(118603, 1.6458882119949068)","(178261, 1.4788933487163227)","(121582, 1.336603799425883)","(143688, 1.2654623239047016)","(105481, 1.2554665743841977)","(154804, 1.2420338979523917)","(175147, 1.232690774849492)","(170816, 1.1578320177852586)","(173567, 1.1307721425045334)","(174056, 1.1093686525998216)"
674336,"(118603, 1.548363131021631)","(154804, 1.2709518400335855)","(143688, 1.232130540030472)","(178261, 1.1536016410603653)","(170816, 0.941122431881753)","(126647, 0.9131142767138687)","(157022, 0.8411615607085423)","(174056, 0.8403389131843833)","(102688, 0.8276800251736496)","(138507, 0.8202217583166982)"
626189,"(118603, 1.2711452369221883)","(143688, 1.0523495650485573)","(154804, 0.9454560610273188)","(174056, 0.8242912921106251)","(178261, 0.798110606638164)","(126647, 0.7727938331000328)","(150279, 0.73891846313371)","(170816, 0.7011556055878758)","(138590, 0.6599729908529735)","(157022, 0.6293929418722368)"
619059,"(118603, 1.1934497628484184)","(143688, 1.1892819387601916)","(174056, 0.9905758475247932)","(154804, 0.9764686625262409)","(126647, 0.7906222924850757)","(157022, 0.7381345013021492)","(138590, 0.7087512154746438)","(138507, 0.6931075291969668)","(173567, 0.6572328322927656)","(170816, 0.6510171731354741)"
643738,"(118603, 1.3950656017515337)","(143688, 1.222391213667421)","(154804, 1.0984466991007498)","(126647, 1.0468868978341295)","(138590, 0.9794796550648717)","(178261, 0.9464330386073245)","(138507, 0.9222540452964842)","(174056, 0.9214924005981504)","(170816, 0.9108163741083914)","(133211, 0.8965050007499298)"
666193,"(118603, 1.5946119785213626)","(143688, 1.309146704444299)","(154804, 1.2913368621183365)","(170816, 1.0876134473367314)","(102688, 0.9654743759573998)","(126647, 0.9491291295047919)","(129371, 0.9433613202943656)","(178261, 0.9064293602204094)","(138507, 0.8974026155988126)","(174056, 0.8922323713283196)"
648147,"(118603, 1.2802999897524154)","(174056, 1.1976727427153224)","(143688, 1.1582011010449396)","(154804, 1.1051902042566049)","(131325, 0.8292767699390828)","(126647, 0.8247530634738935)","(138507, 0.8234583375238269)","(129631, 0.818597261374088)","(170816, 0.8182293746646687)","(178261, 0.7992422658076668)"
615974,"(118603, 1.3756138608332544)","(143688, 1.1504191507163744)","(154804, 0.9913270683046784)","(157022, 0.9150337264074493)","(126647, 0.888673685849168)","(175147, 0.8778138554143069)","(156981, 0.8030231461813411)","(129371, 0.757060803841175)","(162832, 0.7256387748025872)","(174056, 0.7116663560378287)"
667773,"(118603, 1.219745692114779)","(143688, 1.1565853251367786)","(138507, 1.0899429536204925)","(154804, 1.0758637352861786)","(174056, 0.875478697919323)","(105481, 0.7637920119984885)","(150279, 0.7516075432971159)","(131877, 0.6813060791317829)","(133160, 0.6736629974188533)","(131325, 0.6190935329491005)"


In [19]:
r_df.shape

(12611, 10)

In [41]:
con_df = pd.melt(r_df.reset_index(), id_vars= 'index' )

In [47]:
r_df.to_csv('../Data/r_df.csv')

NameError: name 'r_df' is not defined

In [31]:
con_df.head(5)

,index,variable,value
0,654353,0,"(118603, 1.6989002978456509)"
1,611604,0,"(118603, 1.6458882119949068)"
2,674336,0,"(118603, 1.548363131021631)"
3,626189,0,"(118603, 1.2711452369221883)"
4,619059,0,"(118603, 1.1934497628484184)"


In [24]:
con_df.to_csv('../Data/con_df.csv', index=False)

In [38]:
con_df.dtypes

index        int64
variable    object
value       object
dtype: object

In [3]:
df_recom_rating =pd.read_csv('C://Users//rupadhyay//Desktop//ML//Analytics Summit Bibliotheca E Book subscription prediction//Data//All.csv',encoding='latin1')

In [4]:
df_recom_rating.head(5)

,USERID,variable,BOOKID,RATING
0,600003,0,118603,1.358360
1,600003,1,143688,1.190312
2,600003,2,150279,1.146495
3,600003,3,154804,1.120519
4,600003,4,138507,1.055322


In [22]:
df_missing =pd.read_csv('C://Users//rupadhyay//Desktop//ML//Analytics Summit Bibliotheca E Book subscription prediction//Data//Missing.csv',encoding='latin1')

In [6]:
df_missing.head(5)

,UserID
0,663059
1,663063
2,663065
3,663072
4,663079


In [9]:
df_top_book = df_recom_rating.groupby(['variable',])['RATING'].max()

In [21]:
#Finding top 10 rated books, to be recommended to users without any purchase
missing_rating = ['118603','150279','178261','154804','174056','175147','143688','138507','105481','126647']

In [23]:
#For single row prediction
append_rating = []
for row in df_missing['UserID']:
    append_rating.append(missing_rating)

In [25]:
df_missing['BookID'] = append_rating

In [32]:
#for multiple row level prediction
#for row in df_missing['UserID']:
#    for a in missing_rating:
#        append_rating.append(a)

In [37]:
len(append_rating)

23890

In [26]:
#df_missing['BookID'] = append_rating
df_missing.shape

(2389, 2)

In [39]:
#for creating multi row prediction
#df_missing = df_missing.iloc[np.arange(len(df_missing)).repeat(10)]

In [41]:
df_missing['BookID'] = append_rating

c:\users\rupadhyay\python 3.6\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [27]:
df_missing

,UserID,BookID
0,663059,"[118603, 150279, 178261, 154804, 174056, 17514..."
1,663063,"[118603, 150279, 178261, 154804, 174056, 17514..."
2,663065,"[118603, 150279, 178261, 154804, 174056, 17514..."
3,663072,"[118603, 150279, 178261, 154804, 174056, 17514..."
4,663079,"[118603, 150279, 178261, 154804, 174056, 17514..."
5,663081,"[118603, 150279, 178261, 154804, 174056, 17514..."
6,663086,"[118603, 150279, 178261, 154804, 174056, 17514..."
7,663090,"[118603, 150279, 178261, 154804, 174056, 17514..."
8,663099,"[118603, 150279, 178261, 154804, 174056, 17514..."
9,663104,"[118603, 150279, 178261, 154804, 174056, 17514..."


In [28]:
#Creating single line csv for missing ratings
df_missing.to_csv('../Data/missing_user_rating.csv', index=False)

In [43]:
df_non_missing = df_recom_rating[['USERID','BOOKID']]

In [44]:
df_non_missing

,USERID,BOOKID
0,600003,118603
1,600003,143688
2,600003,150279
3,600003,154804
4,600003,138507
5,600003,170816
6,600003,126647
7,600003,174056
8,600003,102688
9,600003,175147


In [45]:
df_non_missing.to_csv('../Data/df_non_missing.csv', index=False)

In [46]:
df_missing.to_csv('../Data/df_missing.csv', index=False)